# LSTM (Long Short Term Memory) 

LSTM is a type of RNN (Recurrent Neural Network) that is capable of learning long-term dependencies. It is widely used in time series forecasting, natural language processing, and other applications.

LSTM是一种RNN（循环神经网络），它能够学习长期依赖关系。它广泛应用于时间序列预测、自然语言处理等应用。

## 1. LSTM Structure

![](https://img-blog.csdnimg.cn/20210327195559364.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

LSTM网络在训练时会使用上一时刻的信息，加上本次时刻的输入信息来共同训练。

举个简单的例子：在第一天我生病了（初始状态$H_0$），然后吃药（利用输入信息$X_1$训练网络），第二天好转但是没有完全好$H_1$，再吃药$X_2$,病情得到好转$H_2$,如此循环往复直到病情好转。因此，输入$X_t$是吃药，时间轴T是吃多天的药，隐含层状态是病情状况。因此我还是我，只是不同状态的我。

上面的图表示包含2个隐含层的LSTM网络，在$T=1$时刻看，它是一个普通的BP网络，在$T=2$时刻看也是一个普通的BP网络，只是沿时间轴展开后，$T=1$训练的隐含层信息H,C会被传递到下一个时刻$T=2$，如下图所示。上图中向右的五个常常的箭头，所的也是隐含层状态在时间轴上的传递。

In the above figure, $H$ represents the hidden layer state, and $C$ is the forget gate.

图中$H$表示隐藏层状态，$C$是遗忘门

实际上遗忘门比作记忆门Remember gate更合适，因为它不是遗忘，而是记忆。因为这一层如果是1，就是记忆，如果是0，就是遗忘。

## 2. LSTM Input Structure

为了更好理解LSTM结构，还必须理解LSTM的数据输入情况。仿照3通道图像的样子，在加上时间轴后的多样本的多特征的不同时刻的数据立方体如下图所示：

![](https://img-blog.csdnimg.cn/20210327201559465.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

右边的图是我们常见模型的输入，比如XGBOOST，lightGBM，决策树等模型，输入的数据格式都是这种（$NF$）的矩阵，而左边是加上时间轴后的数据立方体，也就是时间轴上的切片，它的维度是（$NT*F$）,第一维度是样本数，第二维度是时间，第三维度是特征数，如下图所示：

![](https://img-blog.csdnimg.cn/20210327201821362.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

这样的数据立方体很多，比如天气预报数据，把样本理解成城市，时间轴是日期，特征是天气相关的降雨风速PM2.5等，这个数据立方体就很好理解了。在NLP里面，一句话会被embedding成一个矩阵，词与词的顺序是时间轴T，索引多个句子的embedding三维矩阵如下图所示：

![](https://img-blog.csdnimg.cn/20210327201900507.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

## 3. LSTM Code by Pytorch


### 3.1 LSTM in Pytorch


pytorch中定义的LSTM模型的参数如下：

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import os
import matplotlib.pyplot as plt

In [ ]:
class torch.nn.LSTM(*args, **kwargs)
"""
参数有：
    input_size:,       #x的特征维度
    hidden_size:,      #隐藏层的特征维度
    num_layers:,       #lstm隐层的层数,默认为1
    bias:              #False则bihbih=0和bhhbhh=0. 默认为True
    batch_first:       #True则输入输出的数据格式为 (batch, seq, feature)
    dropout:           #除最后一层,每一层的输出都进行dropout,默认为: 0
    bidirectional:     #True则为双向lstm默认为False
"""

- input_size：x的特征维度，就是数据立方体中的F，在NLP中就是一个词被embedding后的向量长度，如下图所示：
![](https://img-blog.csdnimg.cn/20210327202457470.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

- hidden_size：隐藏层的特征维度（隐藏层神经元个数），如下图所示，我们有两个隐含层，每个隐藏层的特征维度都是5。注意，非双向LSTM的输出维度等于隐藏层的特征维度。

![](https://img-blog.csdnimg.cn/20210327202534441.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

- num_layers：lstm隐层的层数，上面的图我们定义了2个隐藏层。

- batch_first：用于定义输入输出维度，后面再讲。

- bidirectional：是否是双向循环神经网络，如下图是一个双向循环神经网络，因此在使用双向LSTM的时候我需要特别注意，正向传播的时候有（$H_t$, $C_t$）,反向传播也有（$H_t^\prime$, $C_t^\prime$）,前面我们说了非双向LSTM的输出维度等于隐藏层的特征维度，而双向LSTM的输出维度是隐含层特征数2，而且$H,C$的维度是时间轴长度2。

### 3.2 输入进LSTM的数据格式 Data Format for LSTM

In [ ]:
input(seq_len, batch, input_size)
"""
参数有：
    seq_len:            #序列长度，在NLP中就是句子长度，一般都会用pad_sequence补齐长度
    batch:              #每次喂给网络的数据条数，在NLP中就是一次喂给网络多少个句子
    input_size:         #特征维度，和前面定义网络结构的input_size一致。

"""

前面也说到，如果LSTM的参数 batch_first=True，则要求输入的格式是：

In [ ]:
#input(batch, seq_len, input_size)


刚好调换前面两个参数的位置。其实这是比较好理解的数据形式，下面以NLP中的embedding向量说明如何构造LSTM的输入。

之前我们的embedding矩阵如下图：

![](https://img-blog.csdnimg.cn/20210327203625378.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

如果把batch放在第一位，则三维矩阵的形式如下：

![](https://img-blog.csdnimg.cn/20210327203644310.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

其转换过程如下图所示：

![](https://img-blog.csdnimg.cn/20210327204302953.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70#pic_center)

LSTM的另外两个输入是 $h_0$ 和 $c_0$，可以理解成网络的初始化参数，用随机数生成即可。

In [ ]:
"""
h0(num_layers * num_directions, batch, hidden_size)
c0(num_layers * num_directions, batch, hidden_size)


参数：
    num_layers：隐藏层数
    num_directions：如果是单向循环网络，则num_directions=1，双向则num_directions=2
    batch：输入数据的batch
    hidden_size：隐藏层神经元个数
"""

注意，如果我们定义的input格式是：

In [ ]:
#input(batch, seq_len, input_size)

则$H$和$C$的格式也是要变的：

In [ ]:
#h0(batch, num_layers * num_directions,  hidden_size)
#c0(batch, num_layers * num_directions,  hidden_size)

### 3.3 LSTM output format 输出的格式

LSTM的输出是一个tuple，如下：

In [ ]:
"""
output,(ht, ct) = net(input)
    output:            #最后一个状态的隐藏层的神经元输出
    ht:                #最后一个状态的隐含层的状态值
    ct:                #最后一个状态的隐含层的遗忘门值
"""

output的默认维度是：

In [ ]:
"""
output(seq_len, batch, hidden_size * num_directions)
ht(num_layers * num_directions, batch, hidden_size)
ct(num_layers * num_directions, batch, hidden_size)
"""

和input的情况类似，如果我们前面定义的input格式是：

则$h_t$和$c_t$的格式也是要变的：

In [ ]:
"""
#若input的维度为
input(batch, seq_len, input_size)
# 则对应的h0和c0的维度为
ht(batc，num_layers * num_directions, h, hidden_size)
ct(batc，num_layers * num_directions, h, hidden_size)
"""

### 3.4 Summary

我们回过头来看看ht和ct在哪里，请看下图：

![](https://img-blog.csdnimg.cn/20210327204947654.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70)

output在哪里？请看下图：

![](https://img-blog.csdnimg.cn/20210327205100524.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70#pic_center)

### 3.5 LSTM combined with other networks LSTM与其他网络的组合

output的维度等于隐藏层神经元的个数，即hidden_size，在一些时间序列的预测中，会在output后，接上一个全连接层，全连接层的输入维度等于LSTM的hidden_size，之后的网络处理就和BP网络相同了，如下图：

![](https://img-blog.csdnimg.cn/20210327205155470.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTc0NDE5Mg==,size_16,color_FFFFFF,t_70#pic_center)

### 3.6 LSTM pytorch实现

In [1]:
#用pytorch实现以上结构

import torch
from torch import nn

class RegLSTM(nn.Module):
    def __init__(self):
        super(RegLSTM, self).__init__()
        # 定义LSTM
        self.rnn = nn.LSTM(input_size, hidden_size, hidden_num_layers)
        # 定义回归层网络，输入的特征维度等于LSTM的输出，输出维度为1
        self.reg = nn.Sequential(
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        x, (ht,ct) = self.rnn(x)
        seq_len, batch_size, hidden_size= x.shape
        x = x.view(-1, hidden_size)
        x = self.reg(x)
        x = x.view(seq_len, batch_size, -1)
        return x


和RNN类似，LSTM也有两种使用方式，直接调用nn.LSTM和详细分步骤的nn.LSTMCell，下面是两种方式的代码：


In [3]:
# 直接调用nn.LSTM，无需分步控制

lstm = nn.LSTM(input_size=100, hidden_size=20, num_layers=4)
print(lstm)

x=torch.randn(10, 3, 100)
out, (h, c) = lstm(x)

print(out.shape, h.shape, c.shape)


LSTM(100, 20, num_layers=4)
torch.Size([10, 3, 20]) torch.Size([4, 3, 20]) torch.Size([4, 3, 20])


第二种实现方式，使用nn.LSTMCell，代码如下：


In [5]:
# 用nn.LSTMCell实现一个one-layer LSTM

print("One layer LSTMcell")

cell = nn.LSTMCell(input_size=100, hidden_size=20)
h = torch.zeros(3, 20)
c = torch.zeros(3, 20)

for xt in x:
    h, c = cell(xt, [h, c])

print(h.shape, c.shape)



#用nn.LSTMCell实现一个two-layer LSTM

print("Two layer LSTMcell")

cell1 = nn.LSTMCell(input_size=100, hidden_size=30)
cell2 = nn.LSTMCell(input_size=30, hidden_size=20)
h1 = torch.zeros(3, 30)
c1 = torch.zeros(3, 30)
h2 = torch.zeros(3, 20)
c2 = torch.zeros(3, 20)

for xt in x:
    h1, c1 = cell1(xt, [h1, c1])
    h2, c2 = cell2(h1, [h2, c2])

print(h2.shape, c2.shape)


One layer LSTMcell
torch.Size([3, 20]) torch.Size([3, 20])
Two layer LSTMcell
torch.Size([3, 20]) torch.Size([3, 20])


## 4. LSTM CASE 实战案例

### 4.1 Sentiment Classification 情感分类

In [1]:
import spacy
from torchtext.legacy import data, datasets



ModuleNotFoundError: No module named 'torchtext.legacy'

In [3]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')


AttributeError: module 'torchtext.data' has no attribute 'Field'